In [107]:
from collections import Counter
import numpy as np
import pandas as pd
import ast

df_init = pd.read_csv("data.csv")

# add Pheromone column in df, it will be updated as the algorithm runs
df_init['Pheromone'] = 1

next_path = [ast.literal_eval(a) for a in df_init['Next Possible Path']]
df_init['Next Possible Path'] = next_path

In [108]:
paths = []
for i in df_init.index:
    path_init = df_init['Possible Path'].iloc[i]
    new_var = path_init.lower()    
    globals()[new_var] = df_init.iloc[i].values.tolist()
    paths.append(df_init.iloc[i].values.tolist())

In [109]:
# Calculate sum of distances and UCS values for all paths
sum_dist = sum(path[2] for path in paths)
sum_ucs = sum(path[3] for path in paths)

# Calculate length for multiple objectives (distance and UCS)
def length(ant):
    summ = 0
    for i in ant:
        for j in paths:
            if i == j[0]:
                sums = (j[2]/sum_dist + j[3]/sum_ucs)/2
                summ += sums
    return summ

In [110]:
# probabiity calculation for each path
def probabs(adja):
    pher_path_quality = []
    for i in adja:
        for j in paths:
            if j[0] == i:
                # Calculate pheromone and path quality for "distance"
                dis_pher_pq = j[4]*(1/(j[2]))
                dis_pher_pq = dis_pher_pq/sum_dist

                # Calculate pheromone and path quality for "ucs"
                ucs_pher_pq= j[4]*(1/(j[3]))
                ucs_pher_pq = ucs_pher_pq/sum_ucs

                pher_pq = (dis_pher_pq + ucs_pher_pq)/2
                pher_path_quality.append(pher_pq)

    # Calculate probability for each path
    summ = sum(pher_path_quality)
    probs = [pher_pq/summ for pher_pq in pher_path_quality]
    
    return probs

In [111]:
# randomly choosing path based on probability roulette
def choose_paths(adja):
    probab = probabs(adja)
    thresholds = np.cumsum(probab)
    r = np.random.random()

    for i, threshold in enumerate(thresholds):
        if r < threshold:
            return adja[i]

    return adja[-1]

In [112]:
# initiate the ant to pass the random path
def ant(init_path_1, init_path_2):
    path = []

    starter = choose_paths([init_path_1[0],init_path_2[0]])
    path.append(starter)

    for i in paths:
        if path[-1] == i[0]:
            adj = i[1]
            if len(adj)==0:
                break
            else:
                adj_random = choose_paths(adj)
                path.append(adj_random)
    return path

In [113]:
# pheromone evaporation
def evaporation(constant):
    for i in paths:
        i[4] = i[4] * (1 - constant)

# pheromone update
def update_pherom(ants):
    for i in ants:
        pherom = 1 / (length(i))
        for j in i:
            for k in paths:
                if k[0] == j:
                    k[4] = k[4] + pherom

In [114]:
# running the algorithm
def full_aco(iter_number, ant_number, init_path_1, init_path_2, evap_constant=0.3):


    for i in range(iter_number):
        evaporation(constant=evap_constant)
        ants = []
        for j in range(ant_number):
            path = ant(init_path_1, init_path_2)
            ants.append(path)
        update_pherom(ants)
    return paths, ants


paths, ants = full_aco(iter_number = 100, ant_number = 500, init_path_1 = ac, init_path_2 = ab) 

In [115]:
print('\nChoosen path after running full algorithm')
print(Counter([str(i) for i in ants]).most_common())


Choosen path after running full algorithm
[("['AC', 'CE', 'EG', 'GI']", 435), ("['AC', 'CD', 'DF', 'FG', 'GI']", 60), ("['AC', 'CD', 'DF', 'FH', 'HI']", 5)]


In [116]:
print('\n All Path after running full algorithm')
for i in paths:
    print(i)


 All Path after running full algorithm
['AB', ['BC', 'BD'], 100, 80, 4.856098385294428]
['AC', ['CD', 'CE'], 80, 83, 6551.735928955854]
['BC', ['CD', 'CE'], 155, 83, 4.856098361211149]
['BD', ['DE', 'DF'], 125, 185, 2.4083279156541963e-08]
['CD', ['DE', 'DF'], 95, 185, 840.3700214546761]
['CE', ['EF', 'EG'], 125, 185, 5716.222005862284]
['DE', ['EF', 'EG'], 130, 185, 3.4440130810237883e-09]
['DF', ['FG', 'FH'], 120, 70, 840.3700214753154]
['EF', ['FG', 'FH'], 195, 70, 1.6605783187457943e-09]
['EG', ['GI'], 130, 50, 5716.222005864069]
['FG', ['GI'], 175, 50, 780.7686150601903]
['FH', ['HG', 'HI'], 120, 60, 59.60140641678492]
['GI', [], 110, 60, 6497.022794069354]
['HG', ['GI'], 150, 50, 0.03217314498808268]
['HI', [], 80, 70, 59.56923327179684]
